# Creating Neural Network by Steps Using TensorFlow 

## Import

In [17]:
import keras
import tensorflow as tf
import matplotlib.pyplot as plt

In [18]:
print(tf.__version__)

2.0.0-dev20191002


## Prepare Data

In [15]:
# Training data
x_data = [
          [1,2,1], [1,3,2], [1,3,4], [1,5,5], 
          [1,7,5], [1,2,5], [1,6,6], [1,7,7]
         ]
y_data = [
          [0,0,1], [0,0,1], [0,0,1], [0,1,0], 
          [0,1,0], [0,1,0], [1,0,0], [1,0,0]
         ]

# Testing data
x_test = [[2,1,1], [3,1,2], [3,3,4]]
y_test = [[0,0,1], [0,0,1], [0,0,1]]

# Placeholder for inputs and labels
X = tf.keras.backend.placeholder([None, 3], dtype="float32")
Y = tf.keras.backend.placeholder([None, 3], dtype="float32")

# Weight
W = tf.Variable(tf.keras.backend.random_normal([3, 3]))
# Bias
b = tf.Variable(tf.keras.backend.random_normal([3]))

## Hypothesis

In [44]:
hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)
hypothesis

<tf.Tensor 'Softmax_3:0' shape=(None, 3) dtype=float32>

## Cost function

In [55]:
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.keras.backend.log(hypothesis), axis=1))
cost

<tf.Tensor 'Mean_3:0' shape=() dtype=float32>

## Optimizer

In [58]:
optimizer = tf.optimizers.Adam(learning_rate=0.01)
optimizer

## Prediction

In [59]:
prediction = tf.argmax(hypothesis, 1)
is_correct = tf.equal(prediction, tf.argmax(Y, 1))

## Accuracy

In [61]:
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
accuracy

<tf.Tensor 'Mean_5:0' shape=() dtype=float32>

## MNIST Database

In [63]:
import tensorflow as tf
# Tensorflow already incldues MNNIST data set
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
import random

# Get input data as one_hot encoding format
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

# Labels: 0 ~ 9
nb_labels = 10

# MNIST data image of shape 28 x 28 = 784
X = tf.placeholder(tf.float32, [None, 784])
# 0 ~ 9 digits recofnition = 10 labels
Y = tf.placeholder(tf.float32, [None, nb_labels])
# Weight
W = tf.Variable(tf.random_normal([784, nb_labels]))
# Bias
b = tf.Variable(tf.random_normal([nb_labels]))

# Hypothesis - softmax
hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)
# Cost
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis =1))
# Optimizer
optimizer = tf.train.GradientDescentOptimizer(\
                    learning_rate=0.1).minimize(cost)

# Test model
is_correct = tf.equal(tf.argmax(hypothesis, 1), \
                        tf.argmax(Y, 1))
# Calculate accuracy
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

# Epoch - How many times data will be trained
training_epochs = 15
# Batch - How many data will be trained at once
batch_size = 100

with tf.Session() as sess:
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())
    
    # training cycle
    for epoch in range(training_epochs):
        avg_cost = 0
        # Iteration - (the number of data) / (batch size).
        max_iteration = int(mnist.train.num_examples / batch_size)
        
        for itr in range(max_iteration):
            batch_xs, batch_ys = \
                    mnist.train.next_batch(batch_size)
            c, _ = sess.run([cost, optimizer], 
                        feed_dict={X: batch_xs, Y: batch_ys})
            avg_cost += c / max_iteration
            
        print("Epoch: {0:4d}, Cost: {1:0.9f}".format(\
                        epoch + 1, avg_cost))
        
    print("Learning finished")
    
    # Test the model using test sets
    # accuracy.eval() == sess.run()
    print("Accuracy: ", accuracy.eval(session=sess, \
                        feed_dict={X: mnist.test.images, \
                                   Y: mnist.test.labels}))
    
    # Get on and predict
    r = random.randint(0, mnist.test.num_examples - 1)
    # mnist.test.labels are one-hot encoded
    print("Label: {0}".format(\
            sess.run(tf.argmax(mnist.test.labels[r:r+1], 1))))
    print("Prediction: {0}".format(\
            sess.run(tf.argmax(hypothesis, 1), \
            feed_dict = {X: mnist.test.images[r:r+1]})))
    plt.imshow(mnist.test.images[r:r+1].reshape(28, 28),
               cmap="Greys", interpolation="nearest")
    plt.show()


ModuleNotFoundError: No module named 'tensorflow.examples.tutorials'

## Single Layer Neural Network for MNIST
By using batch with size 100, row of input layer and output layer is 100

In [64]:
import numpy as np

# Single layer neural network
class SingleLayerNeuralNetwork():
    def __init__(self, input_size, \
                       output_size, \
                       weight_init_std=0.01):
        # Dictionary for weights and bias
        self._params = {}
        # randn returns a sample \
        #  from the standard normal distribution which is 1.
        # Multipying with weight_init_std makes \
        #  random number array \
        #  whose standard variation is weight_init_std.
        self._params['W'] = weight_init_std * \
                    np.random.randn(input_size, output_size)
        # Set zero to all biases
        self._params["b"] = np.zeros(output_size)

    # Sigmoid function as an active function
    def sigmoid(self, X):
        return 1 / (1 + np.exp(-X))

    # Softmax function as an output function
    def softmax(self, X):
        ret = None
        if x.ndim == 2:
            X = X.T
            X = X - np.max(X, axis=0)
            Y = np.exp(x) / np.sum(np.exp(X), axis=0)
            ret = Y.T
        else:
            # To avoid overflow
            X = X - np.max(X)
            ret = np.exp(X) / np.sum(np.exp(X))
    
        return ret
    
    # Cross entropy error function
    def cross_entropy_error(self, Y, labels): 
        # Translate one-hot encoded labels to answer index.
        labels = labels.argmax(axis=1)
        
        batch_size = Y.shape[0]
        log_val = np.log(Y[np.arange(batch_size), labels])
        return -np.sum(log_val) / batch_size

    # Prediction function
    def predict(self, X):
        W = self._params["W"]
        b = self._params["b"]

        # Logit
        logit = np.dot(X, W) + b
        hypothesis = self.sigmoid(logit)
        output = self.softmax(hypothesis)

        return output

    # Cost function
    def cost(self, X, labels):
        Y = self.predict(X)

        return self.cross_entropy_error(Y, labels)

    # Accuracy function
    def accuracy(self, X, labels):
        Y = self.predict(X)
        Y = np.argmax(Y, axis=1)
        labels = np.argmax(labels, axis=1)

        accuracy = np.sum(Y == labels) / float(X.shape[0])
        return accuracy

    # Numerical gradient descent function
    def numerical_gradient(self, f, X):
        h = 1e-4
        # Create array which is filled 
        #  with zero and whose size is the same as X
        grad = np.zeros_like(X)

        # Create iterator for numpy arrady
        it = np.nditer(X, \
                       flags=["multi_index"], \
                       op_flags=["readwrite"])
        while not it.finished:
            idx = it.multi_index
            tmp_val = X[idx]

            X[idx] = float(tmp_val) + h
            fxh1 = f(X)

            X[idx] = tmp_val - h
            fxh2 = f(X)

            grad[idx] = (fxh1 - fxh2) / (2 * h)

            X[idx] = tmp_val
            it.iternext()

        return grad

    # Gradient function
    def gradient(self, X, labels):
        cost_W = lambda W: self.cost(X, labels)

        grads = {}
        grads["W"] = self.numerical_gradient(cost_W, self._params["W"])
        grads["b"] = self.numerical_gradient(cost_W, self._params["b"])

        return grads
    
    # Update Weight and bias
    def update(self, grad, learning_rate):
        for key in ("W", "b"):
            self._params[key] -= learning_rate * grad[key]
